# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [13]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [14]:
sdc=school_data_complete

## District Summary

In [15]:
# Calculate the total number of unique schools
sdc.school_name.nunique()

15

In [16]:
# Calculate the total number of students
sdc.student_name.count()

39170

In [17]:
# Calculate the total budget
sdc.groupby('school_name').budget.max().sum()

24649428

In [18]:
sdc.head(2)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635


In [19]:
# Calculate the average (mean) math score
sdc.math_score.mean()

78.98537145774827

In [20]:
# Calculate the average (mean) reading score
sdc.reading_score.mean()

81.87784018381414

In [21]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
sdc[sdc.math_score>69].student_name.count()/sdc.student_name.count()*100

74.9808526933878

In [22]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
sdc[sdc.reading_score>69].student_name.count()/sdc.student_name.count()*100


85.80546336482001

In [23]:
# Use the following to calculate the percentage of students that passed math and reading
sdc[(sdc.math_score>=70)&(sdc.reading_score>=70)].student_name.count()/sdc.student_name.count()*100

65.17232575950983

In [24]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schools":[sdc.school_name.nunique()],
                                "Total Students":[sdc.student_name.count()],
                                "Total Budget":[sdc.groupby('school_name').budget.max().sum()],
                                "Average Math Score":[sdc.math_score.mean()],
                                "Average Reading Score":[sdc.reading_score.mean()],
                                "% Passing Math":[sdc[sdc.math_score>69].student_name.count()/sdc.student_name.count()*100],
                                "% Passing Reading":[sdc[sdc.reading_score>69].student_name.count()/sdc.student_name.count()*100],
                                "% Overall Passing":[sdc[(sdc.math_score>=70)&(sdc.reading_score>=70)].student_name.count()/sdc.student_name.count()*100]})
                                                        

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [25]:
# Use the code provided to select the school type

school_types = school_data.set_index(["school_name"])["type"]

In [26]:
# Calculate the total school budget and per capita spending
print(f'Total School Budget: ${sdc.groupby("school_name").budget.max().sum():,}')
print(f'Per Capita: ${sdc.groupby("school_name").budget.max().sum()/sdc.student_name.count():,.2f}')

Total School Budget: $24,649,428
Per Capita: $629.29


In [55]:
# Calculate the average test scores
school_type = sdc.groupby("school_name").type.unique()
total_per_school_budget = sdc.groupby("school_name").budget.mean()
per_school_average_math = sdc.groupby("school_name").math_score.sum()
per_school_average_reading = sdc.groupby("school_name").reading_score.sum()
per_school_total_student= sdc.groupby("school_name").student_name.count()
per_student_budget = total_per_school_budget/per_school_total_student
per_school_math = sdc[sdc.math_score>=70].groupby("school_name").student_name.count()
per_school_reading = sdc[sdc.reading_score>=70].groupby("school_name").student_name.count()
per_school_reading_math =sdc[(sdc.reading_score>=70)&(sdc.math_score>=70)].groupby("school_name").student_name.count()
percent_passing_math = per_school_math/per_school_total_student*100
percent_passing_reading =per_school_reading/per_school_total_student*100
percent_passing_overall = per_school_reading_math/per_school_total_student*100

grp = sdc.groupby('school_name')
mPc = sdc[sdc.math_score>69].groupby('school_name').student_name.count()
rPc = sdc[sdc.reading_score>69].groupby('school_name').student_name.count()
aPc = sdc[(sdc.reading_score>69)&(sdc.math_score>69)].groupby('school_name').student_name.count()


school_summary = pd.DataFrame({ "School type":school_type,
                                "Total Students":(per_school_total_student).map('{:,}'.format),
                                 "Total School Budget":(total_per_school_budget).map('${:,.2f}'.format),
                                "Per Student Budget":(per_student_budget).map('{:.0f}'.format),
                               "Average Math Score": (grp.math_score.mean()).map('{:.3f}'.format),
                               "Average Reading Score": (grp.reading_score.mean()).map('{:.3f}'.format),
                               "% Passing Math": (mPc/grp.student_name.count()*100).map('{:.3f}%'.format),
                               "% Passing Reading": (rPc/grp.student_name.count()*100).map('{:.3f}%'.format),
                               "% Overall Passing": (aPc/grp.student_name.count()*100).map('{:.3f}%'.format)
                             })
school_summary


,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,[District],"4,976","$3,124,928.00",628,77.048,81.034,66.680%,81.933%,54.642%
Cabrera High School,[Charter],"1,858","$1,081,356.00",582,83.062,83.976,94.133%,97.040%,91.335%
Figueroa High School,[District],"2,949","$1,884,411.00",639,76.712,81.158,65.988%,80.739%,53.204%
Ford High School,[District],"2,739","$1,763,916.00",644,77.103,80.746,68.310%,79.299%,54.290%
Griffin High School,[Charter],"1,468","$917,500.00",625,83.351,83.817,93.392%,97.139%,90.599%
Hernandez High School,[District],"4,635","$3,022,020.00",652,77.290,80.934,66.753%,80.863%,53.528%
Holden High School,[Charter],427,"$248,087.00",581,83.803,83.815,92.506%,96.253%,89.227%
Huang High School,[District],"2,917","$1,910,635.00",655,76.629,81.183,65.684%,81.316%,53.514%
Johnson High School,[District],"4,761","$3,094,650.00",650,77.072,80.966,66.058%,81.222%,53.539%


## Highest-Performing Schools (by % Overall Passing)

In [62]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
school_summary.sort_values('% Overall Passing', ascending=False)[:5]

,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,[Charter],"1,858","$1,081,356.00",582,83.062,83.976,94.133%,97.040%,91.335%
Thomas High School,[Charter],"1,635","$1,043,130.00",638,83.418,83.849,93.272%,97.309%,90.948%
Griffin High School,[Charter],"1,468","$917,500.00",625,83.351,83.817,93.392%,97.139%,90.599%
Wilson High School,[Charter],"2,283","$1,319,574.00",578,83.274,83.989,93.868%,96.540%,90.583%
Pena High School,[Charter],962,"$585,858.00",609,83.840,84.045,94.595%,95.946%,90.541%


## Bottom Performing Schools (By % Overall Passing)

In [65]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
school_summary.sort_values('% Overall Passing')[:5]

,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,[District],"3,999","$2,547,363.00",637,76.843,80.745,66.367%,80.220%,52.988%
Figueroa High School,[District],"2,949","$1,884,411.00",639,76.712,81.158,65.988%,80.739%,53.204%
Huang High School,[District],"2,917","$1,910,635.00",655,76.629,81.183,65.684%,81.316%,53.514%
Hernandez High School,[District],"4,635","$3,022,020.00",652,77.290,80.934,66.753%,80.863%,53.528%
Johnson High School,[District],"4,761","$3,094,650.00",650,77.072,80.966,66.058%,81.222%,53.539%


## Math Scores by Grade

In [70]:
g = '9th'

In [74]:
sdc[sdc.grade==g].groupby('school_name').math_score.mean()

school_name
Bailey High School       77.083676
Cabrera High School      83.094697
Figueroa High School     76.403037
Ford High School         77.361345
Griffin High School      82.044010
Hernandez High School    77.438495
Holden High School       83.787402
Huang High School        77.027251
Johnson High School      77.187857
Pena High School         83.625455
Rodriguez High School    76.859966
Shelton High School      83.420755
Thomas High School       83.590022
Wilson High School       83.085578
Wright High School       83.264706
Name: math_score, dtype: float64

In [75]:
# Use the code provided to separate the data by grade
# Group by "school_name" and take the mean of each.
# Use the code to select only the `math_score`.
# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
# Minor data wrangling
# Display the DataFrame

math_score_by_grades = pd.DataFrame()
for g in sdc.grade.unique():
    math_score_by_grades[g] = sdc[sdc.grade==g].groupby('school_name').math_score.mean()
math_score_by_grades

,9th,12th,11th,10th
school_name,,,,
Bailey High School,77.083676,76.492218,77.515588,76.996772
Cabrera High School,83.094697,83.277487,82.765560,83.154506
Figueroa High School,76.403037,77.151369,76.884344,76.539974
Ford High School,77.361345,76.179963,76.918058,77.672316
Griffin High School,82.044010,83.356164,83.842105,84.229064
Hernandez High School,77.438495,77.186567,77.136029,77.337408
Holden High School,83.787402,82.855422,85.000000,83.429825
Huang High School,77.027251,77.225641,76.446602,75.908735
Johnson High School,77.187857,76.863248,77.491653,76.691117


## Reading Score by Grade 

In [76]:
# Use the code provided to separate the data by grade
# Group by "school_name" and take the mean of each.                
# Minor data wrangling
# Display the DataFrame
reading_score_by_grades = pd.DataFrame()
for r in sdc.grade.unique():
    reading_score_by_grades[r] = sdc[sdc.grade==r].groupby("school_name").reading_score.mean()
reading_score_by_grades

,9th,12th,11th,10th
school_name,,,,
Bailey High School,81.303155,80.912451,80.945643,80.907183
Cabrera High School,83.676136,84.287958,83.788382,84.253219
Figueroa High School,81.198598,81.384863,80.640339,81.408912
Ford High School,80.632653,80.662338,80.403642,81.262712
Griffin High School,83.369193,84.013699,84.288089,83.706897
Hernandez High School,80.866860,80.857143,81.396140,80.660147
Holden High School,83.677165,84.698795,83.815534,83.324561
Huang High School,81.290284,80.305983,81.417476,81.512386
Johnson High School,81.260714,81.227564,80.616027,80.773431


## Scores by School Spending

In [78]:
school_summary[:2]

,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,[District],"4,976","$3,124,928.00",628,77.048,81.034,66.680%,81.933%,54.642%
Cabrera High School,[Charter],"1,858","$1,081,356.00",582,83.062,83.976,94.133%,97.040%,91.335%


In [90]:
per_student_budget

school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [79]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [80]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = school_summary.copy()

school_name
Bailey High School       $585-630
Cabrera High School         <$585
Figueroa High School     $630-645
Ford High School         $630-645
Griffin High School      $585-630
Hernandez High School    $645-680
Holden High School          <$585
Huang High School        $645-680
Johnson High School      $645-680
Pena High School         $585-630
Rodriguez High School    $630-645
Shelton High School      $585-630
Thomas High School       $630-645
Wilson High School          <$585
Wright High School          <$585
dtype: category
Categories (4, object): ['<$585' < '$585-630' < '$630-645' < '$645-680']

In [96]:
school_spending_df.dtypes

School type                        object
Total Students                     object
Total School Budget                object
Per Student Budget                 object
Average Math Score                 object
Average Reading Score              object
% Passing Math                     object
% Passing Reading                  object
% Overall Passing                  object
Spending Ranges (Per Student)    category
dtype: object

In [93]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_student_budget,bins=spending_bins,labels=labels)

In [94]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

/var/folders/np/_jtx6yzd4qx0w2k45y6xlkdw0000gn/T/ipykernel_44890/4149390191.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]


KeyError: 'Average Math Score'

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`


# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = 
type_reading_scores = 
type_passing_math = 
type_passing_reading = 
type_overall_passing = 

# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`


# Display results
type_summary